In [1]:
# Import Post 2000 Water Use Data

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Set up the WebDriver (e.g., using Chrome)
executable_path = r'C:/Users/sydne/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'

# Create a Service object
service = Service(executable_path)
driver = webdriver.Chrome(service = service) 


# Create df to store extracted data
use2_df = pd.DataFrame()

for i in range(2000, 2001):

    # Open the URL
    driver.get("https://www3.twdb.texas.gov/apps/reports/WU_REP/SumFinal_RegionReportWithReuse")  # Replace with the URL of the page you're scraping

    # Wait for page to load
    wait = WebDriverWait(driver, 3)

    # Wait for the page to load
    time.sleep(3)  

    # Switch to the iframe (inline frame)
    iframe = driver.find_element(By.ID, "reportFrame")
    driver.switch_to.frame(iframe)

    # Locate the dropdown (Replace 'your_dropdown_id' with the actual ID)
    dropdown = Select(driver.find_element(By.ID, 'rptViewer_ctl08_ctl03_ddValue'))

    # Switch the value in the dropdown (e.g., selecting an option by visible text)
    dropdown.select_by_visible_text(str(i))  # Replace with the value you want to select

    # Wait a moment for the page to update after the dropdown change
    time.sleep(2)  

    # Locate the "View Report" button and click it (Replace with the actual button's identifier)
    view_report_button = driver.find_element(By.ID, 'rptViewer_ctl08_ctl00')  # Replace with the actual ID of the button
    view_report_button.click()


    #  Wait for the report and table to be available
    report_area = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, 'rptViewer_ctl13_ReportArea')))
    #table = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, '')))                                                   COULDN'T FIND CORRESPONDING HTML COMPONENT FOR TABLE
    
    # After the dropdown is selected, get the page source
    page_source = driver.page_source

    # Use BeautifulSoup to parse the page source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract the desired data (example: extracting a table)
    table = soup.find('table') 

   # Define the column names you're looking for
    target_columns = ['Munipical Groundwater', 'Municipal Surface Water', 'Manufacturing Groundwater', 'Manufacturing Surface Water', 'Mining Groundwater', 'Mining Surface Water', 'Power Groundwater', 'Power Surface Water', 'Irrigation Groundwater', 'Irrigation Surface Water', 'Livestock Groundwater', 'Livestock Surface Water']

        
    # Filter headers to keep only target columns
    

    # Get rows
    rows = table.find_all('tr')

    # Extract headers and last row
    headers = [header.get_text(strip=True) for header in rows[0].find_all('td')]
    last_row = rows[-1].find_all('td')

    # Extract data from last row
    selected_data = [data.get_text(strip=True) for header, data in zip(headers, last_row) if header in selected_headers]
    
    # Append to dataframe
    use2_df = pd.concat([use2_df, pd.DataFrame(selected_data)], axis = 0)



    # Extract data from the specified columns
    


# Close the browser
driver.quit()


NameError: name 'pd' is not defined

In [ ]:
# Download all temp / precip data    {Not working bc not finding download button}
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import shutil  # For file renaming

def download_and_rename_noaa_data(start_year, end_year, download_dir):
    """
    Downloads NOAA data for a range of years using Selenium and renames the files.

    Args:
        start_year (int): The starting year.
        end_year (int): The ending year.
        download_dir (str): The directory to save the downloaded files.
    """

    # Ensure the download directory exists
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    # Set Chrome options to download files to the specified directory
    chrome_options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": download_dir}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=chrome_options)

    url = "https://www.ncdc.noaa.gov/cdo-web/datasets/GSOM/stations/GHCND:USW00013958/detail"

    driver.get(url)

    for year in range(start_year, end_year + 1):
        try:
            # Select the year
            year_select = Select(driver.find_element(By.ID, "yearSelect"))
            year_select.select_by_value(str(year))

            # Click "View Data" using the data-bind attribute
            view_data_button = driver.find_element(By.XPATH, "//button[@data-bind='click: submit']")
            view_data_button.click()

            # Wait for the download button to appear (adjust time as needed)
            time.sleep(5)

            # Click the download button
            download_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//cr-icon-button[@id='download']")))
            download_button.click()

            # Wait for download to complete (adjust time as needed)
            time.sleep(10)

            # Rename the downloaded file
            original_file_path = os.path.join(download_dir, "data.csv")  # Default filename
            new_file_path = os.path.join(download_dir, f"noaa_data_{year}.csv")

            # Check if the file exists before renaming
            if os.path.exists(original_file_path):
                shutil.move(original_file_path, new_file_path)
                print(f"Downloaded and renamed: {new_file_path}")
            else:
                print(f"Warning: File for year {year} was not found.")

            # Go back to the initial page for the next year
            driver.get(url)

        except Exception as e:
            print(f"Error downloading or renaming data for year {year}: {e}")

    driver.quit()

# Example usage:
start_year = 1970
end_year = 2020
download_directory = r"C:\Users\sydne\Downloads\water_scarcity" #replace with your download path.

download_and_rename_noaa_data(start_year, end_year, download_directory)

Error downloading or renaming data for year 1970: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF717804C25+3179557]
	(No symbol) [0x00007FF7174688A0]
	(No symbol) [0x00007FF7172F91CA]
	(No symbol) [0x00007FF71734FA67]
	(No symbol) [0x00007FF71734FC9C]
	(No symbol) [0x00007FF7173A3627]
	(No symbol) [0x00007FF717377C6F]
	(No symbol) [0x00007FF7173A02F3]
	(No symbol) [0x00007FF717377A03]
	(No symbol) [0x00007FF7173406D0]
	(No symbol) [0x00007FF717341983]
	GetHandleVerifier [0x00007FF7178667CD+3579853]
	GetHandleVerifier [0x00007FF71787D1D2+3672530]
	GetHandleVerifier [0x00007FF717872153+3627347]
	GetHandleVerifier [0x00007FF7175D092A+868650]
	(No symbol) [0x00007FF717472FFF]
	(No symbol) [0x00007FF71746F4A4]
	(No symbol) [0x00007FF71746F646]
	(No symbol) [0x00007FF71745EAA9]
	BaseThreadInitThunk [0x00007FF8F977259D+29]
	RtlUserThreadStart [0x00007FF8FA76AF38+40]

Error downloading or renaming data for year 1971: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF717804C25+3179557]
	(N

Selenium is a framework that is used to control web browers programatically. In this case, it is needed to adjust a drop down on a webpage while scraping data using Beautiful Soup. 
- Web Driver: A low level interface for providing virtual input actions to a web browser
- Inline Frame: (iframe), a webpage embedded within another webpage, allows you to display external content like another webpage, video, or document inside the current page

